# ConvLSTM Model

In [1]:
import torch
import pandas as pd
import numpy as np

# 加载 X 数组
X_loaded = np.load('X_array.npy')

# 加载 Y 数据
Y_loaded = pd.read_pickle('Y_series.pkl')

In [2]:
X_loaded.shape

(400, 400, 5)

In [3]:
Y_loaded.shape

(1795,)

In [4]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset, random_split

# 假设 X_original 是形状为 [400, 400, 5] 的空间数据
# 假设 Y_original 是形状为 [1795,] 的温度时间序列数据
X_original = X_loaded  # 形状: [400, 400, 5]
Y_original = Y_loaded  # 形状: [1795,]

seq_length = 20  # 设定时间序列的长度

# 将时间序列拆分为输入序列和预测值 (滑动窗口方法)
def create_sequences(data, seq_length):
    X_seq = []
    Y_seq = []
    for i in range(len(data) - seq_length):
        X_seq.append(data[i:i+seq_length])  # 输入序列：长度为 seq_length 的窗口
        Y_seq.append(data[i+seq_length])  # 目标：下一个时间步
    return np.array(X_seq), np.array(Y_seq)


In [5]:
# 拆分时间序列数据
Y_seq, Y_target = create_sequences(Y_original, seq_length)
Y_seq = np.expand_dims(Y_seq, axis=-1)  # [1795-seq_length, seq_length, 1] -> [1775, 20, 1]
Y_target = np.expand_dims(Y_target, axis=-1)  # [1795-seq_length, 1] -> [1775, 1]


In [9]:
# 转换为 PyTorch 张量
X_land = torch.tensor(X_original, dtype=torch.float32).unsqueeze(0).repeat(len(Y_seq), 1, 1, 1)  # 形状 [1775, 400, 400, 5]
Y_seq = torch.tensor(Y_seq, dtype=torch.float32)  # 形状 [1775, 20, 1]
Y_target = torch.tensor(Y_target, dtype=torch.float32)  # 形状 [1775, 1]


In [10]:
X_land.shape, Y_seq.shape, Y_target.shape

(torch.Size([1775, 400, 400, 5]),
 torch.Size([1775, 20, 1]),
 torch.Size([1775, 1]))

In [ ]:
# 调整 X_land 的形状以适应 ConvLSTM 的输入格式
X_land = X_land.permute(0, 3, 1, 2).unsqueeze(1).repeat(1, seq_length, 1, 1, 1)  # 形状 [1775, 20, 5, 400, 400]


In [11]:
X_land.shape

torch.Size([1775, 400, 400, 5])

In [ ]:
# 创建 DataLoader
dataset = TensorDataset(X_land, Y_seq, Y_target)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)


In [4]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset, random_split

# 假设 X_original 是形状为 [400, 400, 5] 的空间数据
# 假设 Y_original 是形状为 [1795,] 的温度时间序列数据
X_original = X_loaded
Y_original = Y_loaded

seq_length = 20  # 设定时间序列的长度

# 将时间序列拆分为输入序列和预测值
def create_sequences(data, seq_length):
    X_seq = []
    Y_seq = []
    for i in range(len(data) - seq_length):
        X_seq.append(data[i:i+seq_length])
        Y_seq.append(data[i+seq_length])
    return np.array(X_seq), np.array(Y_seq)

# 拆分时间序列数据
Y_seq, Y_target = create_sequences(Y_original, seq_length)
Y_seq = np.expand_dims(Y_seq, axis=-1)  # [1795-seq_length, seq_length, 1]
Y_target = np.expand_dims(Y_target, axis=-1)  # [1795-seq_length, 1]

# 转换为 PyTorch 张量
X_land = torch.tensor(X_original, dtype=torch.float32).unsqueeze(0).repeat(Y_seq.shape[0], 1, 1, 1)
Y_seq = torch.tensor(Y_seq, dtype=torch.float32)
Y_target = torch.tensor(Y_target, dtype=torch.float32)

# 调整 X_land 的形状以适应 ConvLSTM 的输入格式
X_land = X_land.permute(0, 3, 1, 2).unsqueeze(1)  # 变成 (batch_size, seq_length, channels, height, width)

# 创建 DataLoader
dataset = TensorDataset(X_land, Y_seq, Y_target)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)


In [5]:
def inspect_first_and_last_batch(dataloader):
    """
    只检查并打印 DataLoader 中第一批和最后一批的数据形状和批大小。
    
    参数:
    dataloader (DataLoader): PyTorch 的 DataLoader 对象。
    
    返回:
    None: 直接打印信息。
    """
    total_batches = len(dataloader)
    first_batch = None
    last_batch = None
    
    print(f"Total Batches: {total_batches}")

    for batch_idx, (X, Y_seq, Y_target) in enumerate(dataloader):
        if batch_idx == 0:
            first_batch = (X, Y_seq, Y_target)
        if batch_idx == total_batches - 1:
            last_batch = (X, Y_seq, Y_target)

    if first_batch:
        X, Y_seq, Y_target = first_batch
        print(f"First Batch (Batch 1):")
        print(f"Batch size (X): {X.shape[0]}")
        print(f"X shape: {X.shape}")
        print(f"Y_seq shape: {Y_seq.shape}")
        print(f"Y_target shape: {Y_target.shape}")

    if last_batch:
        X, Y_seq, Y_target = last_batch
        print(f"\nLast Batch (Batch {total_batches}):")
        print(f"Batch size (X): {X.shape[0]}")
        print(f"X shape: {X.shape}")
        print(f"Y_seq shape: {Y_seq.shape}")
        print(f"Y_target shape: {Y_target.shape}")


In [6]:
inspect_first_and_last_batch(train_loader)

Total Batches: 45
First Batch (Batch 1):
Batch size (X): 32
X shape: torch.Size([32, 1, 5, 400, 400])
Y_seq shape: torch.Size([32, 20, 1])
Y_target shape: torch.Size([32, 1])

Last Batch (Batch 45):
Batch size (X): 12
X shape: torch.Size([12, 1, 5, 400, 400])
Y_seq shape: torch.Size([12, 20, 1])
Y_target shape: torch.Size([12, 1])


In [7]:
inspect_first_and_last_batch(val_loader)

Total Batches: 12
First Batch (Batch 1):
Batch size (X): 32
X shape: torch.Size([32, 1, 5, 400, 400])
Y_seq shape: torch.Size([32, 20, 1])
Y_target shape: torch.Size([32, 1])

Last Batch (Batch 12):
Batch size (X): 3
X shape: torch.Size([3, 1, 5, 400, 400])
Y_seq shape: torch.Size([3, 20, 1])
Y_target shape: torch.Size([3, 1])


In [8]:
class ConvLSTMCell(nn.Module):
    def __init__(self, input_dim, hidden_dim, kernel_size, bias=True):
        super(ConvLSTMCell, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.kernel_size = kernel_size
        self.padding = tuple(k // 2 for k in kernel_size)  # 对每个维度分别计算 padding
        self.bias = bias

        self.conv = nn.Conv2d(in_channels=self.input_dim + self.hidden_dim,
                              out_channels=4 * self.hidden_dim,
                              kernel_size=self.kernel_size,
                              padding=self.padding,
                              bias=self.bias)

    def forward(self, input_tensor, cur_state):
        h_cur, c_cur = cur_state

        combined = torch.cat([input_tensor, h_cur], dim=1)  # concatenate along channel axis
        combined_conv = self.conv(combined)
        cc_i, cc_f, cc_o, cc_g = torch.split(combined_conv, self.hidden_dim, dim=1)
        i = torch.sigmoid(cc_i)
        f = torch.sigmoid(cc_f)
        o = torch.sigmoid(cc_o)
        g = torch.tanh(cc_g)

        c_next = f * c_cur + i * g
        h_next = o * torch.tanh(c_next)

        return h_next, c_next

    def init_hidden(self, batch_size, image_size):
        height, width = image_size
        return (torch.zeros(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device),
                torch.zeros(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device))


class ConvLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, kernel_size, num_layers, batch_first=False, bias=True):
        super(ConvLSTM, self).__init__()
        self._check_kernel_size_consistency(kernel_size)
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.kernel_size = kernel_size
        self.num_layers = num_layers
        self.batch_first = batch_first
        self.bias = bias

        cell_list = []
        for i in range(self.num_layers):
            cur_input_dim = self.input_dim if i == 0 else self.hidden_dim[i - 1]
            cell_list.append(ConvLSTMCell(input_dim=cur_input_dim,
                                          hidden_dim=self.hidden_dim[i],
                                          kernel_size=self.kernel_size[i],
                                          bias=self.bias))

        self.cell_list = nn.ModuleList(cell_list)

    def forward(self, input_tensor, hidden_state=None):
        print(f'start ConvLSTM forward pass...')
        if not self.batch_first:
            input_tensor = input_tensor.permute(1, 0, 2, 3, 4)

        b, _, _, h, w = input_tensor.size()
        if hidden_state is None:
            hidden_state = self._init_hidden(batch_size=b, image_size=(h, w))

        layer_output_list = []
        last_state_list = []

        seq_len = input_tensor.size(1)
        cur_layer_input = input_tensor

        for layer_idx in range(self.num_layers):
            h, c = hidden_state[layer_idx]
            output_inner = []
            for t in range(seq_len):
                h, c = self.cell_list[layer_idx](input_tensor=cur_layer_input[:, t, :, :, :], cur_state=[h, c])
                output_inner.append(h)
                print(f'Layer {layer_idx+1}/{self.num_layers}, Step {t+1}/{seq_len} - Hidden state shape: {h.shape}')

            layer_output = torch.stack(output_inner, dim=1)
            cur_layer_input = layer_output

            layer_output_list.append(layer_output)
            last_state_list.append([h, c])
            
        print(f'ConvLSTM forward pass completed.')

        return layer_output_list, last_state_list

    def _init_hidden(self, batch_size, image_size):
        init_states = []
        for i in range(self.num_layers):
            init_states.append(self.cell_list[i].init_hidden(batch_size, image_size))
        return init_states

    @staticmethod
    def _check_kernel_size_consistency(kernel_size):
        if not (isinstance(kernel_size, tuple) or isinstance(kernel_size, list)):
            raise ValueError('`kernel_size` must be a tuple or list')
        if isinstance(kernel_size, list) and len(kernel_size) != len(kernel_size):
            raise ValueError('Inconsistent list length.')

class ConvLSTMModel(nn.Module):
    def __init__(self, seq_length, input_dim, hidden_dim, kernel_size, num_layers):
        super(ConvLSTMModel, self).__init__()
        self.conv_lstm = ConvLSTM(input_dim=input_dim, hidden_dim=hidden_dim, kernel_size=kernel_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim[-1] * 400 * 400, 1)

    def forward(self, x):
        print(f'start ConvLSTMModel forward pass...')
        b, t, c, h, w = x.size()
        print(f'Input shape: {x.shape}')
        
        conv_lstm_out, _ = self.conv_lstm(x)
        print(f'conv_lstm_out[-1].shape: {conv_lstm_out[-1].shape}')
        print(f'conv_lstm_out[-1][:, -1, :, :, :]: {conv_lstm_out[-1][:, -1, :, :, :].shape}')
        last_time_step = conv_lstm_out[-1][:, -1, :, :, :]
        
        print(f'Last time step shape: {last_time_step.shape}')
        
        last_time_step = last_time_step.view(last_time_step.size(0), -1)  # flatten
        output = self.fc(last_time_step)
        print(f'Output shape: {output.shape}')
        print(f'ConvLSTMModel forward pass completed.')
        return output


# 逐步训练模型

In [9]:
# 初始化并训练模型
conv_lstm_model = ConvLSTMModel(seq_length=seq_length, input_dim=5, hidden_dim=[64, 128], kernel_size=[(3, 3), (3, 3)], num_layers=2)

In [10]:
model = conv_lstm_model
train_loader = train_loader
val_loader = val_loader
num_epochs = 2

In [11]:
print('Start training...')
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


Start training...


In [12]:
epoch = 0


In [ ]:
print(f'Epoch {epoch+1}/{num_epochs}')
model.train()
print('Training...')
train_loss = 0.0
for batch_idx, batch in enumerate(train_loader):

    X, _, Y = batch
    optimizer.zero_grad()

    print(f'start training batch {batch_idx+1}/{len(train_loader)}, X shape: {X.shape}, Y shape: {Y.shape}')
    output = model(X)

    print(f'start calculating loss...')
    loss = criterion(output, Y)

    print(f'start backpropagation...')
    loss.backward()

    print(f'start updating weights...')
    optimizer.step()
    train_loss += loss.item()


Epoch 1/2
Training...
start training batch 1/45, X shape: torch.Size([32, 1, 5, 400, 400]), Y shape: torch.Size([32, 1])
Input shape: torch.Size([32, 1, 5, 400, 400])
Layer 1/2, Step 1/1 - Hidden state shape: torch.Size([32, 64, 400, 400])
Layer 2/2, Step 1/1 - Hidden state shape: torch.Size([32, 128, 400, 400])
Last time step shape: torch.Size([32, 128, 400, 400])
Output shape: torch.Size([32, 1])
start calculating loss...
start backpropagation...


In [ ]:
val_loss = 0.0
model.eval()
print('Validating...')
with torch.no_grad():
    for batch_idx, batch in enumerate(val_loader):
        X, _, Y = batch
        print(f'start validating batch {batch_idx+1}/{len(val_loader)}, X shape: {X.shape}, Y shape: {Y.shape}')
        output = model(X)

        print(f'start calculating loss...')
        loss = criterion(output, Y)
        val_loss += loss.item()

print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss/len(train_loader):.4f}, Val Loss: {val_loss/len(val_loader):.4f}')

In [ ]:
for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')
    model.train()
    print('Training...')
    train_loss = 0.0
    for batch_idx, batch in enumerate(train_loader):

        X, _, Y = batch
        optimizer.zero_grad()

        print(f'start training batch {batch_idx+1}/{len(train_loader)}, X shape: {X.shape}, Y shape: {Y.shape}')
        output = model(X)

        print(f'start calculating loss...')
        loss = criterion(output, Y)

        print(f'start backpropagation...')
        loss.backward()

        print(f'start updating weights...')
        optimizer.step()
        train_loss += loss.item()

    val_loss = 0.0
    model.eval()
    print('Validating...')
    with torch.no_grad():
        for batch_idx, batch in enumerate(val_loader):
            X, _, Y = batch
            print(f'start validating batch {batch_idx+1}/{len(val_loader)}, X shape: {X.shape}, Y shape: {Y.shape}')
            output = model(X)

            print(f'start calculating loss...')
            loss = criterion(output, Y)
            val_loss += loss.item()

    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss/len(train_loader):.4f}, Val Loss: {val_loss/len(val_loader):.4f}')

In [9]:
def train_model(model, train_loader, val_loader, num_epochs=10):
    print('Start training...')
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        model.train()
        print('Training...')
        train_loss = 0.0
        for batch_idx, batch in enumerate(train_loader):
            
            X, _, Y = batch
            optimizer.zero_grad()
            
            print(f'start training batch {batch_idx+1}/{len(train_loader)}, X shape: {X.shape}, Y shape: {Y.shape}')
            output = model(X)
            
            print(f'start calculating loss...')
            loss = criterion(output, Y)
            
            print(f'start backpropagation...')
            loss.backward()
            
            print(f'start updating weights...')
            optimizer.step()
            train_loss += loss.item()

        val_loss = 0.0
        model.eval()
        print('Validating...')
        with torch.no_grad():
            for batch_idx, batch in enumerate(val_loader):
                X, _, Y = batch
                print(f'start validating batch {batch_idx+1}/{len(val_loader)}, X shape: {X.shape}, Y shape: {Y.shape}')
                output = model(X)
                
                print(f'start calculating loss...')
                loss = criterion(output, Y)
                val_loss += loss.item()

        print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss/len(train_loader):.4f}, Val Loss: {val_loss/len(val_loader):.4f}')

In [10]:
# 初始化并训练模型
conv_lstm_model = ConvLSTMModel(seq_length=seq_length, input_dim=5, hidden_dim=[64, 128], kernel_size=[(3, 3), (3, 3)], num_layers=2)

In [ ]:
train_model(conv_lstm_model, train_loader, val_loader, num_epochs=2)

Start training...
Epoch 1/2
Training...
start training batch 1/45, X shape: torch.Size([32, 1, 5, 400, 400]), Y shape: torch.Size([32, 1])
Input shape: torch.Size([32, 1, 5, 400, 400])
Layer 1/2, Step 1/1 - Hidden state shape: torch.Size([32, 64, 400, 400])
